In [14]:
import plotly.plotly as py
from plotly.graph_objs import *

import numpy as np
import pandas as pd
import requests
import copy
import googlemaps

# add your google maps api key here
import os
my_google_maps_api_key = os.environ['gmapsapi']
df = pd.read_csv('NJ_supermarkets.csv',encoding='utf-8')

df.head()

gmaps = googlemaps.Client(key=my_google_maps_api_key)

In [15]:
df.head()

,Store #,Address,State,Zip,Phone,store
0,0,45 Vervalen Street Closter,NJ,07624,(201) 367-9099,Whole Foods Market
1,1,1558 North Kings Highway Cherry Hill,NJ,08034,(856) 651-5271,Whole Foods Market
2,2,471 State Route 35 North Middletown,NJ,07701-5036,(732) 758-1688,Whole Foods Market
3,3,319 Chimney Rock Road Bridgewater,NJ,08805,(732) 584-1960,Whole Foods Market
4,4,645 Middlesex Avenue Metuchen,NJ,08840,(732) 515-6180,Whole Foods Market


In [61]:
def parse_geojson(out):
    out = out[0]
    county = out['address_components'][3]['long_name']

    try:
        zipcode = "{}-{}".format(out['address_components'][6]['long_name'],out['address_components'][7]['long_name'])
    except:
        zipcode = out['address_components'][6]['long_name']
    
    formatted_address = out['formatted_address']
    try:
        latitude,longitude = out['geometry']['location']['lat'], out['geometry']['location']['lng']
    except:
        latitude,longitude = 0,0
    
    try:
        boundaries = out['geometry']['bounds']
    except:
        boundaries = 'empty'
    
    place_id = out['place_id']
    return (county,zipcode,formatted_address,latitude,longitude,boundaries,place_id)

In [62]:
parse_geojson(out2)

('Cherry Hill',
 'United States-08034',
 '1558 Kings Hwy N, Cherry Hill, NJ 08034, USA',
 39.9158817,
 -75.0114246,
 'empty',
 'ChIJq2MzcXPLxokRzTxFuzzhcLY')

In [50]:
out2 = gmaps.geocode("1558 North Kings Highway Cherry Hill NJ 08034")
out2

[{'address_components': [{'long_name': '1558',
    'short_name': '1558',
    'types': ['street_number']},
   {'long_name': 'Kings Highway North',
    'short_name': 'Kings Hwy N',
    'types': ['route']},
   {'long_name': 'Erlton-Ellisburg',
    'short_name': 'Erlton-Ellisburg',
    'types': ['neighborhood', 'political']},
   {'long_name': 'Cherry Hill',
    'short_name': 'Cherry Hill',
    'types': ['locality', 'political']},
   {'long_name': 'Camden County',
    'short_name': 'Camden County',
    'types': ['administrative_area_level_2', 'political']},
   {'long_name': 'New Jersey',
    'short_name': 'NJ',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'United States',
    'short_name': 'US',
    'types': ['country', 'political']},
   {'long_name': '08034', 'short_name': '08034', 'types': ['postal_code']}],
  'formatted_address': '1558 Kings Hwy N, Cherry Hill, NJ 08034, USA',
  'geometry': {'location': {'lat': 39.9158817, 'lng': -75.0114246},
   'location

In [63]:
df.head()

,Store #,Address,State,Zip,Phone,store
0,0,45 Vervalen Street Closter,NJ,07624,(201) 367-9099,Whole Foods Market
1,1,1558 North Kings Highway Cherry Hill,NJ,08034,(856) 651-5271,Whole Foods Market
2,2,471 State Route 35 North Middletown,NJ,07701-5036,(732) 758-1688,Whole Foods Market
3,3,319 Chimney Rock Road Bridgewater,NJ,08805,(732) 584-1960,Whole Foods Market
4,4,645 Middlesex Avenue Metuchen,NJ,08840,(732) 515-6180,Whole Foods Market


In [66]:
dg = df #backup!

In [97]:
del df
df = dg

In [147]:
# add geo related field placeholders to dataframe
df["county"] = ''
df["zipcode"] = ''
df["formatted_address"] = ''
df["latitude"] = 0.00
df["longitude"] = 0.00
#df["boundaries"] = ''
df["place_id"] = ''

df['geodict'] = ''

In [148]:
for i,_ in df.iterrows():
    print ("Supermarket #{0}: Getting GPS data for:\n{1}\n".format(i,_.Address + ' ' + _.State + ' ' + _.Zip))
    geodata = gmaps.geocode(_.Address + ' ' + _.State + ' ' + _.Zip)
    try:
        county,zipcode,formatted_address,latitude,longitude,boundaries,place_id = parse_geojson(geodata)
        df.loc[i,'county'] = county
        df.loc[i,'zipcode'] = zipcode
        df.loc[i,'formatted_address'] = formatted_address
        df.loc[i,'latitude'] = latitude
        df.loc[i,'longitude'] = longitude
        #df.loc[i,'boundaries'] = boundaries
        df.loc[i,'place_id'] = place_id
        print ("Added values: \n----------------------------------------------------------------\n{}\n----------------------------------------------------------------\n".format(df.loc[i,'county':]))
    except:
        print ("Issue parsing results - adding to geodict field")
        df.loc[i,'geodict'] = geodata

Supermarket #0: Getting GPS data for:
45 Vervalen Street Closter NJ 07624

Added values: 
----------------------------------------------------------------
county                                        Bergen County
zipcode                                               07624
formatted_address    45 Vervalen St, Closter, NJ 07624, USA
latitude                                            40.9706
longitude                                          -73.9563
boundaries                                                 
place_id                        ChIJNcH9qmvuwokRqwGT5e1Ktp8
geodict                                                    
Name: 0, dtype: object
----------------------------------------------------------------

Supermarket #1: Getting GPS data for:
1558 North Kings Highway Cherry Hill NJ 08034

Added values: 
----------------------------------------------------------------
county                                                Cherry Hill
zipcode                                      

ValueError: Must have equal len keys and value when setting with an iterable

In [151]:
for i,_ in df.iterrows():
    if i >57:
        print ("Supermarket #{0}: Getting GPS data for:\n{1}\n".format(i,_.Address + ' ' + _.State + ' ' + _.Zip))
        geodata = gmaps.geocode(_.Address + ' ' + _.State + ' ' + _.Zip)
        try:
            county,zipcode,formatted_address,latitude,longitude,boundaries,place_id = parse_geojson(geodata)
            df.loc[i,'county'] = county
            df.loc[i,'zipcode'] = zipcode
            df.loc[i,'formatted_address'] = formatted_address
            df.loc[i,'latitude'] = latitude
            df.loc[i,'longitude'] = longitude
            #df.loc[i,'boundaries'] = boundaries
            df.loc[i,'place_id'] = place_id
            print ("Added values: \n----------------------------------------------------------------\n{}\n----------------------------------------------------------------\n".format(df.loc[i,'county':]))
        except:
            print ("Issue parsing results - adding to geodict field")
            df.loc[i,'geodict'] = """{}""".format(geodata)

Supermarket #58: Getting GPS data for:
State Highway 35 And New Bed Rd Belmar NJ 07719

Issue parsing results - adding to geodict field
Supermarket #59: Getting GPS data for:
980 Shrewsbury Ave Eatontown NJ 07724

Added values: 
----------------------------------------------------------------
county                                                 Monmouth County
zipcode                                                          07724
formatted_address         980 Shrewsbury Ave, Eatontown, NJ 07724, USA
latitude                                                       40.3159
longitude                                                     -74.0698
boundaries                                                            
place_id             Eiw5ODAgU2hyZXdzYnVyeSBBdmUsIEVhdG9udG93biwgTk...
geodict                                                               
Name: 59, dtype: object
----------------------------------------------------------------

Supermarket #60: Getting GPS data for:
2101 Stat

In [158]:
ls

 NJ_supermarkets.csv   notebooks/  'Untitled Folder'/   Untitled.ipynb   work/


In [159]:
df.to_csv('NJ_supermarkets_gps.csv',encoding='utf-8',index=False)